# Challenge 01 - Prompt Engineering

Quick tip! To view the Table of Contents for this Notebook in VS Code or within Codespaces, take a look at the "Explorer" tab, expand the "Outline" section. 

## 1. Parameter Experimentation
Let's first set up the Challenge. These cells install the required Python packages, load the environment variables, and relevant Python libraries using the cells below.

### 1.1 Preparation
#### 1.1.1 Setup


**NOTE:** Only one model is used by the Jupyter notebook, which is specified in the .env file. To complete this challenge, use the latest model as instructed by your coach. If you are doing this independently, feel free to use one of the latest Azure OpenAI GPT models.

**NOTE** Double check these values in your .env file to ensure the notebooks run seamlessly. 
* CHAT_MODEL_NAME must equal the deployed model's name in the AI Foundry portal.
* EMBEDDING_MODEL_NAME must equal the deployed embedding model's name in the AI Foundry portal.

####  Set up Azure OpenAI Environment

In [1]:
%pip install -r ../requirements.txt

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import openai
import os
import json

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider


In [3]:
token_provider = get_bearer_token_provider(
    DefaultAzureCredential(),
    "https://cognitiveservices.azure.com/.default"
)

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"

openai.api_base = RESOURCE_ENDPOINT
openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
openai.azure_ad_token_provider = token_provider
chat_model=os.getenv("CHAT_MODEL_NAME")

client = AzureOpenAI(
    azure_endpoint=RESOURCE_ENDPOINT,
    azure_ad_token_provider=token_provider,
    api_version=os.getenv("OPENAI_API_VERSION")
)


#### 1.1.2 Parameter Explanation
We specify the hyperparameters for the Azure OpenAI models within the helper functions. Users can tune the parameters according to different needs.


###### **Temperature**
Temperature ranges from 0 to 2.
Here is a quick breakdown of how it works:
- Low temperature (0 to 0.3): More focused, coherent, and conservative outputs.
- Medium temperature (0.3 to 0.7): Balanced creativity and coherence.
- High temperature (> 0.7): Highly creative and diverse, but potentially less coherent.

###### **Top_p**
Sets the probability mass cutoff for token sampling, affecting the breadth of options the AI considers. 
Higher values lead to more randomness, while lower values result in more focused outputs.

The “top_p” parameter is like a filter that controls how many different words or phrases the language model considers when it’s trying to predict the next word. If you set the “top p” value to 0.5, the language model will only consider the 50 most likely words or phrases that might come next. But if you set the “top p” value to 0.9, the language model will consider the 90 most likely words or phrases.

And as "top_p" and "temperature" performs similar job as hyperparameter, we usually only tune one of them instead of both.

###### **Max_tokens**
Max_tokens determine the maximum length of the generated text. By setting a limit, you can control how much text the LLM model will return, making sure it doesn't give too long of an answer.

###### **Frequency_penalty**
Frequency_penalty makes sure that the text that is generated is varied by giving a penalty to tokens that have already been used in the response.

It ranges from -2.0 to 2.0, with higher values resulting in more diverse output.

Positive values penalize new tokens based on their existing frequency in the text so far, decreasing the model's likelihood to repeat the same line verbatim.

#### 1.1.3 Helper Function
For this hack, make sure you use the latest model as instructed by your coach. If you are doing this independently, feel free to use one of the latest Azure OpenAI GPT models.

This helper function will make it easier to use prompts and look at the generated outputs.

**get_chat_completion** helps create the OpenAI response using the chat model of your choice.

**get_completion_from_messages** helps create the OpenAI response using the chat model of your choice, enabling chat history.


In [47]:
def get_chat_completion(prompt, model=chat_model):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=chat_model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
        max_tokens = 200,
        top_p = 1.0
    )
    return response.choices[0].message.content

In [48]:
def get_completion_from_messages(messages, model=chat_model, temperature=0):
    response = client.chat.completions.create(
        model=chat_model,
        messages=messages,
        temperature=temperature # this is the degree of randomness of the model's output
    )

    return response.choices[0].message.content


#### Try out helper functions

In [7]:
messages =  f"""
tell me a joke.
"""
response = get_chat_completion(messages)
print(response)

Why don't skeletons fight each other?

They don't have the guts!


In [50]:
messages =  [      
    {'role':'user', 'content':'tell me a joke'},   
    {'role':'assistant', 'content':'Why did the chicken cross the road'},   
    {'role':'user', 'content':'I don\'t know'}
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

To get to the other side!


#### Student Tasks (open questions):
1. Create a completion function for a higher temperature between (0,1).
2. Create a completion function for a lower max_token for shorter response.
3. Create acompletion functions with 2 different diversity penalty values between (0,2).

Try out the completion functions you create on the previous case, compare the result you get.

In [16]:
# Try out a completion function for a higher temperature between (0,1)
def get_chat_completion(prompt, temp, model=chat_model):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=chat_model,
        messages=messages,
        temperature=temp, # this is the degree of randomness of the model's output
        max_tokens = 200,
        top_p = 1.0
    )
    return response.choices[0].message.content

messages =  f"""
tell me a joke about dogs, make it like it was an story.
"""
response = get_chat_completion(prompt=messages, temp= 1)
print(response)

Once upon a time in the quaint town of Barksville, there lived a dog named Max. Max was not just any ordinary dog; he was the town's self-appointed mayor, squirrel-chaser, and unofficial alarm clock. Every morning, like clockwork, Max would stroll through the neighborhood, barking to greet every sunrise.

One day, a new family moved into town, bringing along a sleek, modern vacuum cleaner that boasted cutting-edge technology and an unerring dedication to cleanliness. The townsfolk heard whispers about this gadget, claiming it was capable of doing the impossible: keeping dog hair away for good. Max, always curious and slightly mischievous, decided to investigate.

As Max approached the newcomers' house, he saw the vacuum diligently going about its task, whirring across the floor. Max puffed up his chest and barked a mighty bark, declaring his challenge to the machine.

The vacuum paused, its sensors picking up this new presence. It changed direction and started moving towards


In [ ]:
# Try out a completion function for a lower max_token for shorter response
# Try out a completion function for a higher temperature between (0,1)
def get_chat_completion(prompt, maxToken, model=chat_model):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=chat_model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
        max_tokens = maxToken,
        top_p = 1.0
    )
    return response.choices[0].message.content

messages =  f"""
tell me a joke about dogs, make it like it was an story.
"""
response = get_chat_completion(prompt=messages, maxToken = 20)
print(response)

Once upon a time in a small, bustling town, there was a dog named Max who fancied


In [32]:
# Try out completion functions with 2 different diversity penalty values between (0,2)
def get_completion_from_messages(messages, penalty, model=chat_model, temperature=0):
    response = client.chat.completions.create(
        model=chat_model,
        messages=messages,
        temperature=temperature,
        frequency_penalty = penalty
    )
    return response.choices[0].message.content


messages =  [      
    {'role':'user', 'content':'What do you think about dot net'},   
    {'role':'assistant', 'content':'Specifically about what topic?'},   
    {'role':'user', 'content':'About the AI integration features'}
]
response = get_completion_from_messages(messages, penalty = 2)
print(response)

.NET provides several features and tools that facilitate the integration of AI into applications. Here are some key aspects:

1. **ML.NET**: This is a cross-platform, open-source machine learning framework for .NET developers. It allows you to build custom machine learning models using C# or F#. ML.NET supports various tasks like classification, regression, clustering, anomaly detection, and more.

2. **Integration with Azure Cognitive Services**: .NET can easily integrate with Azure Cognitive Services to add pre-built AI capabilities such as vision recognition (e.g., image tagging), speech processing (e.g., text-to-speech), language understanding (e.g., sentiment analysis), and decision-making APIs.

3. **ONNX Runtime Support**: The Open Neural Network Exchange (ONNX) format is supported in .NET through the ONNX Runtime library which allows developers to run pre-trained deep learning models across different platforms efficiently.

4. **TensorFlow.NET & SciSharp Stack**: For those who 

### 1.2 System Message Engineering
Users can achieve the response from models in their desired tone through adjusting the system message.

#### 1.2.1 Change of Tone

In [37]:
messages =  [      
    {'role':'assistant', 'content':'How can I help you?'},
    {'role':'user', 'content':'tell me a joke'}
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Why don't skeletons fight each other?

They don't have the guts!


In [38]:
messages =  [  
    {'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
    {'role':'assistant', 'content':'How can I help you?'},
    {'role':'user', 'content':'tell me a joke'}
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Verily, why did the chicken cross the road? To escape the foul deeds of the poultry-geist that plagued yon barnyard!


#### Student Task (open question):
Make the assistant tell the joke in the tone of your favorite character by editing the system message.


In [42]:
# Make the assistant tell the joke in the tone of your favorite character by editing the system message.
messages =  [  
    {'role':'system', 'content':'You are an assistant that speaks like Cristiano Ronaldo.'},    
    {'role':'assistant', 'content':'How can I help you?'},
    {'role':'user', 'content':'What is your fav drink?'}
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Listen, my friend, I'm all about staying in top shape, so I keep it simple. Water is my go-to drink. It's essential for hydration and keeping the body feeling good. But, of course, I also enjoy a nice fresh juice now and then, especially a good orange juice, you know? It's all about balance!


#### 1.2.2 Remind the Company Name

In [43]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},  
{'role':'user', 'content':'Hi, my name is Mel.'},
{'role':'assistant', 'content': "Hi! It's nice to meet you. \
Is there anything I can help you with today?"},  
{'role':'user', 'content':'Yes, can you remind me which company I work for?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I'm sorry, but I don't have access to personal data such as your employment details. You might want to check your email signature, any company documents, or your LinkedIn profile for that information. If there's anything else I can do to help, just let me know!


#### Student Task: 
Make the assistant give a reliable reminder of company name by giving context in the system message.

In [45]:
# Make the assistant give a reliable reminder of company name by giving context in system message.
messages =  [  
{'role':'system', 'content':'You are friendly chatbot designed by Microsoft for only internal employee use.'},  
{'role':'user', 'content':'Hi, my name is Mel. I am a software engineer in the most amazing company'},
{'role':'assistant', 'content': "Hi! It's nice to meet you. \
Is there anything I can help you with today?"},  
{'role':'user', 'content':'Yes, can you remind me which company I work for?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Of course! You work for Microsoft, the most amazing company. 😊 How can I assist you today?


## 2. Iterative Prompting Principles
It is generally a good practice to perform iterative prompting so the model could generate the most appropriate response given the user's specification. 
- **Principle 1: Write clear and specific instructions**
- **Principle 2: Give the model time to “think”**



### 2.1 Write clear and specific instructions

#### Tactic 1: Delimiters

#### Student Task:
Use delimiters to clearly indicate distinct parts of the input

Delimiters can be anything like: ````, """, < >, `<tag> </tag>`, `:`

In [ ]:
text = f"""
    The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). However, the event retained the Tokyo 2020 branding for marketing purposes. It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. The Games were the most expensive ever, with total spending of over $20 billion.The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, South Korea and preceding the 2022 Winter Olympics in Beijing, China. Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.\nNew events were introduced in existing sports, including 3x3 basketball, freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program.The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. Great Britain finished fourth, with a total of 22 gold and 64 medals. The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.'
"""

In [46]:
# Use delimiters to clearly indicate distinct parts of the input, and ask the model to summarize the text.
text = f"""
    The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as: "Tokyo 2020", was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan. With some preliminary events that began on 21 July 2021.
    Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). However, the event retained the Tokyo 2020 branding for marketing purposes. 
    It was largely held behind closed doors with no public spectators permitted due to the declaration of a <tag>state of emergency</tag> in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. The Games were the most expensive ever, with total spending of over $20 billion.The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, South Korea and preceding the 2022 Winter Olympics in Beijing, China. Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.\nNew events were introduced in existing sports, including 3x3 basketball, freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program.The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. Great Britain finished fourth, with a total of 22 gold and 64 medals. The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.'
"""

#### Tactic 2: Summarization: specify word counts, extract information

#### Text to summarize

In [ ]:
game_review = """
The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, \
was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, \
with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, \
on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, \
the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). \
However, the event retained the Tokyo 2020 branding for marketing purposes. \
It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. \
The Games were the most expensive ever, with total spending of over $20 billion. \
The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), \
and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. \
The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, \
South Korea and preceding the 2022 Winter Olympics in Beijing, China. \
Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.
New events were introduced in existing sports, including 3x3 basketball, \
freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. \
New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. \
The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, \
sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program. \
The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). \
Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. \
Great Britain finished fourth, with a total of 22 gold and 64 medals. \
The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. \
Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.
"""
messages = [      
    {'role':'system', 'content':'You are a helpful assistant.'}, 
    {'role':'user', 'content':'I need you to summarize a text.'},   
    {'role':'assistant', 'content':'Please provide the text you would like me to summarize.'},   
    {'role':'user', 'content': game_review} 
]
response = get_completion_from_messages(messages, temperature=1)
print(response)



The 2020 Summer Olympics, also known as Tokyo 2020, were held in Tokyo, Japan, from 23 July to 8 August 2021, after being postponed due to the COVID-19 pandemic. Originally set for 2020, the games were rescheduled but retained their "Tokyo 2020" branding. For safety, the Olympics were held mostly without spectators, marking a significant shift in how the event was conducted. These Games, costing over $20 billion, were the most expensive in history and the first to occur without official public attendance. 

Tokyo became the first Asian city to host the Summer Olympics twice, and these Games were part of three consecutive Olympics in East Asia. The 2020 Olympics were unique as they occurred in an odd-numbered year and non-leap year, a first since 1900. New events included 3x3 basketball, freestyle BMX, and mixed-gender team events. Japan, as host, added baseball, softball, karate, sport climbing, surfing, and skateboarding to the program, with the latter three making their debut.

The U

#### Student Task:
Summarize with a focus on the special parts of the 2020 Summer Olympics and with a word limit

In [ ]:
# Summarize with a focus on the special parts of the 2020 Summer Olympics and with a word limit
game_review = """
The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, \
was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, \
with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, \
on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, \
the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). \
However, the event retained the Tokyo 2020 branding for marketing purposes. \
It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. \
The Games were the most expensive ever, with total spending of over $20 billion. \
The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), \
and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. \
The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, \
South Korea and preceding the 2022 Winter Olympics in Beijing, China. \
Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.
New events were introduced in existing sports, including 3x3 basketball, \
freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. \
New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. \
The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, \
sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program. \
The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). \
Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. \
Great Britain finished fourth, with a total of 22 gold and 64 medals. \
The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. \
Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.
"""
messages = [      
    {'role':'system', 'content':'You are a helpful assistant.'}, 
    {'role':'user', 'content':'I need you to summarize a text. focus on the special parts of the 2020 Summer Olympics.'},   
    {'role':'assistant', 'content':'Please provide the text you would like me to summarize.'},   
    {'role':'user', 'content': game_review} 
]


def get_completion_from_messages_with_max_tokens(messages, maxTokens, model=chat_model, temperature=0):
    response = client.chat.completions.create(
        model=chat_model,
        messages=messages,
        max_tokens= maxTokens,
        temperature=temperature # this is the degree of randomness of the model's output
    )

    return response.choices[0].message.content

response = get_completion_from_messages_with_max_tokens(messages, maxTokens=50, temperature=1)
print(response)



The 2020 Summer Olympics, also known as Tokyo 2020, took place in Tokyo, Japan, from 23 July to 8 August 2021, having been postponed from 2020 due to the COVID-19 pandemic. Un


#### Student Task:
Try "extract" instead of "summarize"

In [ ]:
# Try "extract" instead of "summarize"
# Summarize with a focus on the special parts of the 2020 Summer Olympics and with a word limit
game_review = """
The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, \
was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, \
with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, \
on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, \
the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). \
However, the event retained the Tokyo 2020 branding for marketing purposes. \
It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. \
The Games were the most expensive ever, with total spending of over $20 billion. \
The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), \
and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. \
The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, \
South Korea and preceding the 2022 Winter Olympics in Beijing, China. \
Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.
New events were introduced in existing sports, including 3x3 basketball, \
freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. \
New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. \
The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, \
sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program. \
The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). \
Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. \
Great Britain finished fourth, with a total of 22 gold and 64 medals. \
The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. \
Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.
"""
messages = [      
    {'role':'system', 'content':'You are a helpful assistant.'}, 
    {'role':'user', 'content':'I need you to extract a text. focus on the special parts of the 2020 Summer Olympics.'},   
    {'role':'assistant', 'content':'Please provide the text you would like me to extract.'},   
    {'role':'user', 'content': game_review} 
]

response = get_completion_from_messages_with_max_tokens(messages, maxTokens=500, temperature=1)
print(response)



Here are some special aspects of the 2020 Summer Olympics (Tokyo 2020):

1. **Postponement**: Originally scheduled from 24 July to 9 August 2020, the Games were postponed to 2021 due to the COVID-19 pandemic, marking the first time the Olympics were rescheduled rather than canceled.

2. **No Spectators**: For the first time, the Olympics were held without public spectators due to a state of emergency in Tokyo.

3. **Branding**: Despite the postponement, the event retained the Tokyo 2020 branding.

4. **Expense**: These were the most expensive Olympic Games ever, costing over $20 billion.

5. **Japan’s Olympic History**: This was the fourth Olympics held in Japan and Tokyo became the first Asian city to host the Summer Olympics twice.

6. **Consecutive East Asian Games**: These Games were part of a series of three consecutive Olympic events hosted in East Asia, bookended by the 2018 Winter Olympics in Pyeongchang and the 2022 Winter Olympics in Beijing.

7. **Unique Timing**: First Olym

#### Tactic 3: Inferring: ask for emotions, sentiment, or topics 

#### Student Task:
Identify types of emotions and sentiment (positive/negative) of the review below

Format in a JSON object

In [78]:
review = """
Philip Barker of Inside the Games opined that for many athletes and supporters, \
the tone of the ceremony was dignified and appropriate. Hashimoto stated in a press interview that the flame would "quietly go out", \
which he felt that "It was an apt description of a dignified and low key Ceremony which conveyed a sense of gratitude that the Games had been able to take place at all."\
Dominic Patten of Deadline Hollywood argued that the ceremony was an "uneven mixtape" of contrasts, \
comparing the low-key "celebration of the culture of the Asian power and brow moping acknowledgement of the pandemic" to the jubilant Paris segment, \
as well as clichÃ©-filled speech of Thomas Bach. Alan Tyres of The Daily Telegraph discussed the IOC updated motto as a sign of things to come. \
He stated, "The updated Olympic motto of 'faster, higher, \
stronger â€“ together' fits with how sport is covered and contextualised at this moment in history: \
inclusion, diversity, justice and a duty of care to the athletes must be taken into consideration as much as performance." \
He also discussed the strangeness of the ceremony, as it was performed without a stadium audience.
"""

messages = [      
    {'role':'system', 'content':'You are a helpful assistant.'}, 
    {'role':'user', 'content':'Detect a sentiment out of the following text'},   
    {'role':'assistant', 'content':'Please provide the text you would like me to analyze.'},   
    {'role':'user', 'content': review} # No curly braces needed here
]

response = get_completion_from_messages_with_max_tokens(messages, maxTokens=500, temperature=1)
print(response)

The sentiment of the text appears to be mixed. 

Philip Barker and Hashimoto express a positive sentiment, highlighting the dignified and appropriate tone of the ceremony, and expressing gratitude that the Games were able to take place.

Dominic Patten, however, provides a more critical view, describing the ceremony as an "uneven mixtape" with contrasts, and pointing out the clichéd nature of Thomas Bach's speech.

Alan Tyres discusses both positive and critical aspects, acknowledging the updated motto's relevance to contemporary values such as inclusion and diversity, while also noting the strangeness of the ceremony due to the absence of a stadium audience.

Overall, the text reflects a blend of appreciation and criticism.


In [ ]:
# Identify types of emotions and sentiment (positive/negative) of the review above. Format in a JSON object.
review = """
Philip Barker of Inside the Games opined that for many athletes and supporters, \
the tone of the ceremony was dignified and appropriate. Hashimoto stated in a press interview that the flame would "quietly go out", \
which he felt that "It was an apt description of a dignified and low key Ceremony which conveyed a sense of gratitude that the Games had been able to take place at all."\
Dominic Patten of Deadline Hollywood argued that the ceremony was an "uneven mixtape" of contrasts, \
comparing the low-key "celebration of the culture of the Asian power and brow moping acknowledgement of the pandemic" to the jubilant Paris segment, \
as well as clichÃ©-filled speech of Thomas Bach. Alan Tyres of The Daily Telegraph discussed the IOC updated motto as a sign of things to come. \
He stated, "The updated Olympic motto of 'faster, higher, \
stronger â€“ together' fits with how sport is covered and contextualised at this moment in history: \
inclusion, diversity, justice and a duty of care to the athletes must be taken into consideration as much as performance." \
He also discussed the strangeness of the ceremony, as it was performed without a stadium audience.
"""

messages = [      
    {'role':'system', 'content':'You are a helpful assistant.'}, 
    {'role':'user', 'content':'Detect a sentiment out of the following text (positive or negative) and do it in json format'},   
    {'role':'assistant', 'content':'Please provide the text you would like me to analyze.'},   
    {'role':'user', 'content': review} 
]

response = get_completion_from_messages_with_max_tokens(messages, maxTokens=500, temperature=1)
print(response)

Based on the provided text, the sentiment is mixed. Various perspectives and opinions are expressed, some highlighting dignity and appropriateness, while others describe contrasts and criticize certain aspects. Here's the sentiment analysis in JSON format:

```json
{
  "sentiment": "mixed"
}
```


#### Student Challenge:
Infer 3 topics of the story below

In [60]:
story = """
The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, \
was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, \
with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, \
on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, \
the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). \
However, the event retained the Tokyo 2020 branding for marketing purposes. \
It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. \
The Games were the most expensive ever, with total spending of over $20 billion. \
The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), \
and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. \
The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, \
South Korea and preceding the 2022 Winter Olympics in Beijing, China. \
Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.
New events were introduced in existing sports, including 3x3 basketball, \
freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. \
New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. \
The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, \
sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program. \
The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). \
Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. \
Great Britain finished fourth, with a total of 22 gold and 64 medals. \
The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. \
Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.
"""


In [ ]:
# Infer 3 topics of the story above. 
messages = [      
    {'role':'system', 'content':'You are a helpful assistant.'}, 
    {'role':'user', 'content':'Inferr 3 topics from the story I am going to share with you'},   
    {'role':'assistant', 'content':'Please, provide the text you would like me to analyze.'},   
    {'role':'user', 'content': story} 
]

response = get_completion_from_messages_with_max_tokens(messages, maxTokens=200, temperature=1)
print(response)

From this story about the 2020 Summer Olympics, three prominent topics can be inferred:

1. **Impact of the COVID-19 Pandemic on the Olympics**: The Tokyo 2020 Olympics were unique due to their postponement from 2020 to 2021 because of the global COVID-19 pandemic. This marked the first time in history that the Olympic Games were postponed rather than canceled, and they were held largely without spectators to ensure safety.

2. **Historical and Regional Significance**: The event was significant for being the most expensive Olympic Games ever and for being the fourth Olympics held in Japan, making Tokyo the first city in Asia to host the Summer Olympics twice. It was part of a consecutive series of Olympics in East Asia, highlighting the region's growing prominence in hosting major international events.

3. **Introduction and Success of New Sports and Participants**: The Tokyo 2020 Olympics saw the introduction of new sports and events, such as 3x3 basketball, freestyle BMX


#### Tactic 4: Transforming: specify target language and writing style, and ask for grammar check

#### Student Task:
Universal Translator

People all over the world want to know the Olympic game news in their native language. In this case, the news needs to be translated into different languages. Translate each news item below into both Korean and English.

In [64]:
news = [
"Palestine received a universality invitation from FINA to send two top-ranked swimmers (one per gender) in their respective individual events to the Olympics, \
based on the FINA Points System of June 28, 2021.",
"比赛包括两轮：预赛和决赛。预赛成绩最好的8次接力队晋级决赛。必要时使用游泳比赛来打破平局以晋级下一轮。"]

In [ ]:
# Write code here for student task
messages = [      
    {'role':'system', 'content':'You are a helpful assistant.'}, 
    {'role':'user', 'content':'Translate the following news for me, make it all korean and all english'},   
    {'role':'assistant', 'content':'Please provide the news you want me to translate.'},   
    {'role':'user', 'content': news[0] + news[1]} 
]

response = get_completion_from_messages_with_max_tokens(messages, maxTokens=150, temperature=1)
print(response)

**English Translation:**

Palestine received a universality invitation from FINA to send two top-ranked swimmers (one per gender) in their respective individual events to the Olympics, based on the FINA Points System of June 28, 2021. The competition includes two rounds: preliminaries and finals. The 8 relay teams with the best results in the preliminaries advance to the finals. If necessary, swim-offs are used to break ties to advance to the next round.

**Korean Translation:**

팔레스타인은 FINA로부터 보편성 초청을 받아 2021년 6월 28일 FINA 포인트 시스템에 따라 올림픽에 각 종목에서 남녀 별로 최고 순


#### Student Task:
Tone Transformation

Writing can vary based on the intended audience. ChatGPT can produce different tones. Transform the following message into a business letter.

In [69]:
original_message = "David, it's John! OMG, the Olympic game is so crazy"

In [ ]:
# Task: write code here for student task
messages = [      
    {'role':'system', 'content':'You are a helpful assistant.'}, 
    {'role':'user', 'content':'Transform the following messsage into a business letter'},   
    {'role':'assistant', 'content':'Please provide the news you want me to convert.'},   
    {'role':'user', 'content': original_message} 
]

response = get_completion_from_messages_with_max_tokens(messages, maxTokens=300, temperature=1)
print(response)

[Your Name]  
[Your Address]  
[City, State, ZIP Code]  
[Email Address]  
[Phone Number]  
[Date]  

David [Last Name]  
[David's Position]  
[David's Company Name]  
[Company Address]  
[City, State, ZIP Code]  

Dear David [Last Name],

I hope this message finds you well. I wanted to take a moment to express my excitement about the ongoing Olympic Games. The events have been absolutely remarkable and thrilling to watch.

I look forward to discussing this further and hearing your thoughts on the games at your convenience.

Best regards,

John [Your Last Name]


#### Student Task:
Format Conversion

ChatGPT can translate between formats. The prompt should describe the input and output formats. Convert the following JSON data into HTML format.

In [72]:
data_json = { "The 2020 Summer Olympics Opening Ceremony audience name list" :[ 
    {"name":"Shyam", "email":"shyamjaiswal@gmail.com"},
    {"name":"Bob", "email":"bob32@gmail.com"},
    {"name":"Jai", "email":"jai87@gmail.com"}
]}

In [77]:
# Write code here for student task
messages = [      
    {'role':'system', 'content':'You are a helpful assistant.'}, 
    {'role':'user', 'content':'Transform the following json into html'},   
    {'role':'assistant', 'content':'Please provide the news you want me to convert.'},   
    {'role':'user', 'content': f"{data_json}"} 
]

response = get_completion_from_messages_with_max_tokens(messages, maxTokens=300, temperature=1)
print(response)

To transform the given JSON into HTML, we will loop through the list of audience members and create an HTML structure to display their names and emails. Here is an example of how you can convert this JSON to HTML:

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>2020 Summer Olympics Opening Ceremony Audience</title>
    <style>
        table {
            width: 50%;
            margin: 20px auto;
            border-collapse: collapse;
        }
        th, td {
            border: 1px solid #ddd;
            padding: 8px;
            text-align: left;
        }
        th {
            background-color: #f2f2f2;
        }
    </style>
</head>
<body>

    <h1>2020 Summer Olympics Opening Ceremony Audience</h1>
    <table>
        <thead>
            <tr>
                <th>Name</th>
                <th>Email</th>
            </tr>
        </thead>
        <tbody>
          

#### Student Task:
Spellcheck and Grammar check the following text. 

To signal to the LLM that you want it to proofread your text, you instruct the model to 'proofread' or 'proofread and correct'.

In [ ]:
check_text = [ 
  "Alongside the main Emblem blue, the five other colors use in the branding of the 2020 Games is : Kurenai red,   Ai blue,   Sakula pink,   Fuji purple, and   Matsuba green.",
  "The competition have three round: heats, semifinals, and a final."
]



In [82]:
# Write code here for student task
combined_text = "\n".join(check_text)
messages = [      
    {'role':'system', 'content':'You are a helpful assistant.'}, 
    {'role':'user', 'content':'Proofread the following text for me please'},   
    {'role':'assistant', 'content':'Please provide the text.'},   
    {'role':'user', 'content': f"{check_text}"} 
]

response = get_completion_from_messages_with_max_tokens(messages, maxTokens=300, temperature=1)
print(response)

Certainly! Here is the proofread version of the text:

"Alongside the main Emblem blue, the five other colors used in the branding of the 2020 Games are: Kurenai red, Ai blue, Sakura pink, Fuji purple, and Matsuba green. The competition has three rounds: heats, semifinals, and a final."


#### Tactic 5: Expanding: customize the automated reply

#### Student Task:
Customize the automated reply to the following customer email.

The customer faced an issue while buying the Olympics game ticket.

In [ ]:
# given the sentiment from the tactic on "inferring",
# and the original customer message, customize the email
sentiment = "negative"

# review for a ticket transaction
review = f"""
I bought the ticket of "Men's 100 metre freestyle swimming" game last week.\
The transaction went through successfully. However, I still have not received the ticket.\
Over one week has passed.\
"""

In [ ]:
# Write code here for student task

#### Tactic 6: Chatbot: personalize conversations for specific tasks or behaviors

#### Student Task:
Create a conversation with chatbot to know where the 2020 Summer Olympics is held.

In [ ]:
# Write code here for student task

### 2.2 Give the model time to “think” 

#### Tactic 1: Specify the steps required to complete a task
Sometimes you can help the model "slow down" and give more robust, detailed answers by specifying the steps it should take.

Let's ask for output in multiple specified formats.

In [ ]:
prompt = f"""
Your task is to help a journalist summarize information from the article for publication.

Write a title based on the information provided in the context delimited by triple backticks. 
The title should be short, catchy, and reflective of the article's narrative.

After the title, generate five keywords from the context.

After the keywords, include a table to organize the information. 
The table should have two columns. In the first column is the title.
In the second column include the keywords as a list.

Give the table the title 'Article Publishing Information'.

Format everything as HTML that can be used in a website.
Place the title in a <div> element.

Context: ```{text}

""" 

get_chat_completion(prompt)

#### Tactic 2: Instruct the model to work out its own solution

There are two main methods we will demonstrate in this section to get the model to work through a problem instead of rushing to a conclusion, chain-of-thought and chaining. The strategies can lead to increased accuracy, detail, and the ability to work through complex challenges.


2.1 - Chain-of-thought prompting
- Ask the model to reason
- One-shot example
- Chatbot reasoning

2.2 - Chaining

Let's continue working with the Olympics dataset.

#### 2.2.1 Chain-of-Thought Prompting

Let's do a bit of math. GPT models occasionally don't do super well on solving direct math problems, so let's walk GPT through the problem.

Let's break down tasks into smaller pieces.

Read more about methods and whitepaper research here: https://github.com/openai/openai-cookbook/blob/main/techniques_to_improve_reliability.md

#### Tactic 1: You can start by specifically asking the model to simply think step-by-step.

In [ ]:
text = f"""
    The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). However, the event retained the Tokyo 2020 branding for marketing purposes. It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. The Games were the most expensive ever, with total spending of over $20 billion.The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, South Korea and preceding the 2022 Winter Olympics in Beijing, China. Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.\nNew events were introduced in existing sports, including 3x3 basketball, freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program.The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. Great Britain finished fourth, with a total of 22 gold and 64 medals. The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.'
"""

# From Azure documentation
prompt = "Who was the most decorated (maximum medals) individual athlete in the Olympic games that were held at Sydney? Take a step-by-step approach in your response, cite sources and give reasoning before sharing final answer in the below format: ANSWER is: <name>"
get_chat_completion(prompt)

In [ ]:
# Another example
prompt = f"""
What is the largest time zone difference is between the top two countries who 
won the most gold medals in the 2020 Tokyo olympics?

Use the context below and think aloud as you solve the problem, step-by-step.

Context: {text}
"""
get_chat_completion(prompt)

#### Tactic 2: One-shot example 
Another common tactic is to provide one example of a query and an ideal response. The model will learn from that example and apply the patterns to a new question.

In [ ]:
# Notice how this response may not be ideal, or the most accurate.
prompt = f"""
    The United States has 113 total medals, 39 of which are gold medals. 
    
    Great Britain has 64 medals and 22 gold medals. 
    
    How many more silver and bronze medals does the United States have over Great Britain?
"""

get_chat_completion(prompt)

In [79]:
# Give the model a one-shot example to solve the question more thoroughly

prompt = f"""
Question: The United States has 113 total medals, 39 of which are gold medals. How many medals are silver or bronze?
Answer:
[Step 1] - There are three types of medals: gold, silver, and bronze
[Step 2] - We know the gold and total counts of medals, so the number of silver and bronze medals is the difference between the gold (39) and total (113) counts. 
            113-39=74, so there are 74 silver and bronze medals combined. The answer is 74.

===

Answer the following question using similar steps above.

Question: China has 89 total medals, 38 of which are gold medals. How many silver and bronze medals do they have?
Answer:
"""

get_chat_completion(prompt)

'[Step 1] - There are three types of medals: gold, silver, and bronze.\n\n[Step 2] - We know the gold and total counts of medals, so the number of silver and bronze medals is the difference between the gold (38) and total (89) counts. \n\n89 - 38 = 51, so there are 51 silver and bronze medals combined. The answer is 51.'

#### Tactic 3: Chatbot chain-of-thought reasoning
You can build in logic using variables so the chatbot can dynamically demonstrate specific ways of thinking about a problem.

**The input place is at the top of the screen. You should see a popup. Type 'quit' if you want to exit.**

In [ ]:
# Ask the bot to help you make a decision such as deciding whether to take a job or choose between restaurants.
# If the model does not immediately respond to your query, wait 1-5 seconds and retype it.
# If it is not allowing you to give an input, restart the kernel in the navigation bar.
# Type "quit" to end the session

context = '''
        You are a decision bot. Your job is to help come to decision by asking a series of questions one at a time and coming to a reasonable decision based on the information provided.

        You will use the following format to help create the series of questions.

        Template: 
        [Problem/Scenario/Question]: [Provide a brief description of the problem, scenario, or question.]

        Chain of thought:

        [Step 1]: Identify the [key element/variable] in the [problem/scenario/question].
        [Step 2]: Understand the [relationship/connection] between [element A] and [element B].
        [Step 3]: [Analyze/Evaluate/Consider] the [context/implication] of the [relationship/connection] between [element A] and [element B].
        [Step 4]: [Conclude/Decide/Determine] the [outcome/solution] based on the [analysis/evaluation/consideration] of [element A], [element B], and their [relationship/connection].
        [Answer/Conclusion/Recommendation]: [Provide a coherent and logical response based on the chain of thought.]

        You will guide the user though a series of questions one at a time. The first question is broad, and they subsequent questions become more specific. 

        Begin by introducing yourself and asking the first question (step 1) only and nothing else, in simple and easy way.
        '''

conversation=[{"role": "system", "content": context}]

while(True):
    print(len(conversation))
    if len(conversation) == 1:
        response = get_completion_from_messages(conversation)
        conversation.append({"role": "assistant", "content": response})
        print("\n" + response + "\n")
    
    user_input = input('Enter your response: ')
    if user_input.lower() == "quit":
        break 
    conversation.append({"role": "user", "content": user_input})
    
    response = get_completion_from_messages(conversation)
    conversation.append({"role": "assistant", "content": response})
    print("\n" + response + "\n")


#### 2.2.2 - Chaining
Similar to some earlier examples, you can use model outputs from previous queries into other queries. We will show you later in the Hack how to do this at scale.

In [83]:
# Extract medal counts for each country from the news article
# Write an article about the summarized information
# Provide a title for the summary

text = f"""
    The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). However, the event retained the Tokyo 2020 branding for marketing purposes. It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. The Games were the most expensive ever, with total spending of over $20 billion.The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, South Korea and preceding the 2022 Winter Olympics in Beijing, China. Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.\nNew events were introduced in existing sports, including 3x3 basketball, freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program.The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. Great Britain finished fourth, with a total of 22 gold and 64 medals. The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.'
"""

prompt = f"""
    Based on the context below, write a JSON object that contains the number of gold and total medals for each country.
    Context: {text}
"""
num_medals_dict = get_chat_completion(prompt)

prompt = f"""
    Write a brief article about the winners and losers of the Olympics based on medal count:
    {num_medals_dict}
"""
summary = get_chat_completion(prompt)
print(summary)

prompt = f"""
    Give the summary a title in 5 words:
    {summary}
"""
title = get_chat_completion(prompt)
print(title)

**Olympic Medal Count: Winners and Losers**

The Olympic Games are a global stage where nations showcase their athletic prowess, and the medal count often serves as a barometer of success. In the latest edition of the Olympics, several countries emerged as winners, while others had more modest showings.

**Winners:**

1. **United States**: The United States once again demonstrated its dominance in the Olympics, topping the medal count with a total of 113 medals, including 39 golds. This impressive haul underscores the country's depth and versatility across a wide range of sports.

2. **China**: China closely followed the United States, securing 38 gold medals and a total of 89. The country's strong performance highlights its continued investment in sports and its ability to excel in various disciplines.

3. **Japan**: As the host nation, Japan had a remarkable showing, winning 27 gold medals and a total of 58. The home advantage, coupled with strong performances in
"Top Olympic Medal C

#### Student Task (Chaining): 

Your job is now to write code that will determine the country that won the most silver and bronze medals combined.

We can see that the model performs poorly on answering the question directly.

In [ ]:
prompt = f"""
    Based on the context, which country had the most silver and bronze medals?
    Context: {text}
"""

get_chat_completion(prompt)

Write prompts in the cell below that will help the model answer the question by breaking down the tasks into different steps. Make sure it only answers with the information it was given. This concept of grounding will be further introduced in Challenge 3.

You should be able to get the model to answer the question in 2-3 steps.

In [ ]:
# Write code here for student task

## Conclusion

In this first challenge, we covered how to interact with the Azure OpenAI for different goals. Hopefully you were able to see the dynamic versatility of the models and how they can be used to solve a variety of problems using different techniques.

We gave the API short pieces of text using fixed variables. In the next set of challenges, you will see how to use the API with larger datasets.